In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F #torch是关于运算的包
import torchvision
from torchvision import datasets,transforms, models #torchvision则是打包了一些数据集
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable 
import time

In [45]:
### 准备数据 ###
def prepare_data():
    print('==> Preparing data..')
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    trainset = torchvision.datasets.CIFAR10(root='./data/data_pytorch', train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
    valset = torchvision.datasets.CIFAR10(root='./data/data_pytorch', train=False, download=True, transform=transform_test)
    valloader = torch.utils.data.DataLoader(valset, batch_size=50, shuffle=False, num_workers=2)
    return trainloader, valloader

In [37]:
trainloader, valloader = prepare_data()

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [46]:
### 定义网络 ###
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [39]:
class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [44]:
net = VGG('VGG16')
#print(net)
x = torch.randn(10,3,32,32)
y = net(x)
#print(y)

In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [57]:
def train(net, device, trainloader, n_epoches):
    '''Train our model.'''
    # define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
    for epoch in range(n_epoches):
        print('Epoch: %d' % epoch)
        train_loss = 0.0
        correct = 0
        total = 0
        for i, data_batch in enumerate(trainloader):
            inputs, labels = data_batch
            inputs, labels = inputs.to(device), labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimze
            outputs = net(inputs)
            print(outputs)
            print(labels)
            return
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # print statistics
            train_loss += loss.item()
            _, predict = outputs.max(1)
            total += labels.size(0)
            correct += predict.eq(labels).sum().item()
            if i%100 == 99: # print every 100 batches
                print('Loss: %.3f | Acc: %.3f %% (%d/%d)' % (train_loss/100,
                    100*correct/total, correct, total))
                train_loss = 0.0
    print('Finish Training')

In [58]:
train(net, device, trainloader, 1)

Epoch: 0
tensor([[ -0.7631,  -4.0398,  -0.7474,  -1.7073,  16.5885,  -2.9221,   0.2001,
           1.8999,  -5.5469,  -1.9875],
        [ -3.7191,   0.1116,   2.6076,  -2.1457,  -2.3835,  -1.8586,  17.3826,
          -5.4249,  -3.5935,   0.2269],
        [ -3.3841,  -4.0257,   2.5052,  -1.0955,   8.0843,   2.4831,  -1.7781,
           4.2227,  -2.5076,  -3.6173],
        [  6.3324,  -1.3585,  -4.7995,  -1.5299,  -3.6718,  -3.1886, -10.0868,
          -0.5161,  10.9555,   8.0773],
        [  6.6694,  23.0939,  -9.1996, -10.1218,  -5.0884,  -6.2984,  -5.1723,
          -4.2948,   4.4596,   6.0197],
        [ -3.4369,  -1.7692,   4.3821,  -0.2962,  -0.5532,   9.4958,   0.8337,
           2.2067,  -5.0549,  -4.8667],
        [ -1.3537,  -1.7984,  -2.7713,   1.8624,  -1.9598,  -1.1147,  16.4356,
          -2.5513,  -7.1465,   0.5658],
        [ -0.8317,   4.4746,  -6.3499,  -6.3717,  -6.4665,  -3.7841,  -3.1704,
           0.3358,   2.1499,  19.6901],
        [ -2.4719,  23.4271, -13.6556, 

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f1fc41532b0>>
Traceback (most recent call last):
  File "/home/lhw/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/lhw/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/lhw/anaconda3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/lhw/anaconda3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/lhw/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/home/lhw/anaconda3/lib/python3.6/multiprocessing/reduction.py", line 182, in recv_handle
    return recvfds(s, 1)[0]
 

In [51]:
def val(net, device, valloader):
    '''Compute the accuracy of our prediction on validation dataset.'''
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predict = outputs.max(1)
            total += labels.size(0)
            correct += (predict == labels).sum().item()
    # print the accuracy
    print('Accuracy of the network on the val images: %.3f %%' % (
        100 * correct / total))

In [52]:
val(net, device, valloader)

Accuracy of the network on the val images: 88.850 %


In [25]:
print(cifar_train)
#print(cifar_test)
print(cifar_train.train_data.shape)
print(type(cifar_train.train_labels), len(cifar_train.train_labels))

Dataset CIFAR10
    Number of datapoints: 50000
    Split: train
    Root Location: ./data/data_pytorch
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None
(50000, 32, 32, 3)
<class 'list'> 50000


In [11]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [17]:
model = models.vgg16(pretrained=True)
for parma in model.parameters():
    parma.requires_grad = False

In [27]:
#print(model)

In [21]:
for parma in model.parameters():
    parma.requires_grad = False

model.classifier = torch.nn.Sequential(torch.nn.Linear(25088, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 10))


In [26]:
#print(model)

In [28]:
#在训练的时候我们可以自己写代码手动遍历数据集，指定batch和遍历方法，
#不过PyTorch提供了一个DataLoader类来方便我们完成这些操作
trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(cifar_test, batch_size=128, shuffle=True)

In [35]:
train(model, 0, trainloader, 100)

Epoch: 0
tensor([[[-0.1216, -0.1294, -0.1294,  ..., -0.1294, -0.1294, -0.1294],
         [-0.1294, -0.1373, -0.1373,  ..., -0.1373, -0.1373, -0.1373],
         [-0.1294, -0.1373, -0.1373,  ..., -0.1373, -0.1373, -0.1373],
         ...,
         [-0.1294, -0.1373, -0.1373,  ..., -0.1373, -0.1373, -0.1373],
         [-0.1294, -0.1373, -0.1373,  ..., -0.1373, -0.1373, -0.1451],
         [-0.1216, -0.1373, -0.1373,  ..., -0.1373, -0.1373, -0.1529]],

        [[ 0.6078,  0.5922,  0.6000,  ...,  0.6000,  0.6000,  0.6000],
         [ 0.5922,  0.5765,  0.5843,  ...,  0.5843,  0.5843,  0.5843],
         [ 0.5922,  0.5843,  0.5843,  ...,  0.5843,  0.5843,  0.5843],
         ...,
         [ 0.5922,  0.5843,  0.5843,  ...,  0.5843,  0.5843,  0.5843],
         [ 0.5922,  0.5843,  0.5843,  ...,  0.5843,  0.5843,  0.6000],
         [ 0.5922,  0.5843,  0.5922,  ...,  0.5843,  0.5843,  0.5922]],

        [[ 0.4902,  0.4745,  0.4824,  ...,  0.4824,  0.4824,  0.4824],
         [ 0.4745,  0.4588,  0.4667,

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [6]:
# optim中定义了各种各样的优化方法，包括SGD
import torch.optim as optim
   
if use_gpu:
    device = torch.device("cuda:0")
    model = model.to(device)

cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters())

print("Start Training...")
for epoch in range(30):
    for param in ["train", "val"]:
        if param == "train":
            model.train = True
        else:
            model.train = False
            
        running_loss = 0.0
        running_correct = 0 
        batch = 0   
    
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss100 += loss.item()
        if i % 100 == 99:
            print('[Epoch %d, Batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, loss100 / 100))
            loss100 = 0.0

print("Done Training!")

In [8]:
# optim中定义了各种各样的优化方法，包括SGD
import torch.optim as optim
# CrossEntropyLoss就是我们需要的损失函数
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

print("Start Training...")
for epoch in range(30):
    # 我们用一个变量来记录每100个batch的平均loss
    loss100 = 0.0
    # 我们的dataloader派上了用场
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss100 += loss.item()
        if i % 100 == 99:
            print('[Epoch %d, Batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, loss100 / 100))
            loss100 = 0.0

print("Done Training!")

Start Training...
[Epoch 1, Batch   100] loss: 2.305
[Epoch 1, Batch   200] loss: 2.306
[Epoch 1, Batch   300] loss: 2.302
[Epoch 1, Batch   400] loss: 2.302
[Epoch 1, Batch   500] loss: 2.302
[Epoch 1, Batch   600] loss: 2.302
[Epoch 1, Batch   700] loss: 2.301
[Epoch 1, Batch   800] loss: 2.300
[Epoch 1, Batch   900] loss: 2.299
[Epoch 1, Batch  1000] loss: 2.297
[Epoch 1, Batch  1100] loss: 2.295
[Epoch 1, Batch  1200] loss: 2.292
[Epoch 1, Batch  1300] loss: 2.288
[Epoch 1, Batch  1400] loss: 2.281
[Epoch 1, Batch  1500] loss: 2.270
[Epoch 2, Batch   100] loss: 2.220
[Epoch 2, Batch   200] loss: 2.174
[Epoch 2, Batch   300] loss: 2.114
[Epoch 2, Batch   400] loss: 2.084
[Epoch 2, Batch   500] loss: 2.067
[Epoch 2, Batch   600] loss: 2.042
[Epoch 2, Batch   700] loss: 1.988
[Epoch 2, Batch   800] loss: 1.983
[Epoch 2, Batch   900] loss: 1.962
[Epoch 2, Batch  1000] loss: 1.963
[Epoch 2, Batch  1100] loss: 1.897
[Epoch 2, Batch  1200] loss: 1.890
[Epoch 2, Batch  1300] loss: 1.865
[E

[Epoch 16, Batch   700] loss: 0.965
[Epoch 16, Batch   800] loss: 0.973
[Epoch 16, Batch   900] loss: 0.979
[Epoch 16, Batch  1000] loss: 1.007
[Epoch 16, Batch  1100] loss: 0.986
[Epoch 16, Batch  1200] loss: 0.975
[Epoch 16, Batch  1300] loss: 0.951
[Epoch 16, Batch  1400] loss: 0.963
[Epoch 16, Batch  1500] loss: 0.964
[Epoch 17, Batch   100] loss: 0.931
[Epoch 17, Batch   200] loss: 0.921
[Epoch 17, Batch   300] loss: 0.929
[Epoch 17, Batch   400] loss: 0.929
[Epoch 17, Batch   500] loss: 0.981
[Epoch 17, Batch   600] loss: 0.947
[Epoch 17, Batch   700] loss: 0.946
[Epoch 17, Batch   800] loss: 0.966
[Epoch 17, Batch   900] loss: 0.937
[Epoch 17, Batch  1000] loss: 0.969
[Epoch 17, Batch  1100] loss: 0.961
[Epoch 17, Batch  1200] loss: 0.942
[Epoch 17, Batch  1300] loss: 0.907
[Epoch 17, Batch  1400] loss: 0.948
[Epoch 17, Batch  1500] loss: 0.948
[Epoch 18, Batch   100] loss: 0.866
[Epoch 18, Batch   200] loss: 0.907
[Epoch 18, Batch   300] loss: 0.919
[Epoch 18, Batch   400] loss

In [9]:
# 构造测试的dataloader
dataiter = iter(testloader)
# 预测正确的数量和总数量
correct = 0
total = 0
# 使用torch.no_grad的话在前向传播中不记录梯度，节省内存
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # 预测
        outputs = net(images)
        # 我们的网络输出的实际上是个概率分布，去最大概率的哪一项作为预测分类
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 64 %
